0 based on this code: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/convolutional_network.py

1. The size of feauture map after pooling
   After two layers of maxpooling (strides), the size of feature map was reduced from 28*28 to 7*7, so
   in the fully connected layer, the size of the input is 7*7*64 ( 64 is the number of features map in the 
                                                              previous layer)

source: http://cs231n.github.io/convolutional-networks/
F: their spatial extent
S: the stride
W2=(W1−F)/S+1W2=(W1−F)/S+1
H2=(H1−F)/S+1

2. Add regularization term to weights variables when defining the variables using tf.get_variable function

In [1]:
from __future__ import print_function

import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
# The data will download to "/tmp/data"
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Parameters
learning_rate = 0.001
training_iters = 4096*10
batch_size = 128
display_step = 5

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)


# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out


init_weights = tf.truncated_normal_initializer(0.0, stddev=0.01)
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)

def make_var(name,shape,initializer=None,regularizer = None):
    return tf.get_variable(name,shape,initializer = initializer, regularizer = regularizer)

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    # 5x5 conv, 32 inputs, 64 outputs
    # fully connected, 7*7*64 inputs, 1024 outputs
    # 1024 inputs, 10 outputs (class prediction)
    
    'wc1': make_var('weights1',[5,5,1,32],init_weights,regularizer=regularizer),
    'wc2': make_var('weights2',[5,5,32,64],init_weights,regularizer=regularizer),
    'wd1': make_var('weights3',[7*7*64,1024],init_weights,regularizer=regularizer),
    'out': make_var('weights4',[1024, n_classes],init_weights,regularizer=regularizer)
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
cost += reg_term

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 640, Minibatch Loss= 3.710795, Training Accuracy= 0.10938
Iter 1280, Minibatch Loss= 2.601734, Training Accuracy= 0.13281
Iter 1920, Minibatch Loss= 2.416785, Training Accuracy= 0.11719
Iter 2560, Minibatch Loss= 2.375685, Training Accuracy= 0.10156
Iter 3200, Minibatch Loss= 2.347625, Training Accuracy= 0.15625
Iter 3840, Minibatch Loss= 2.347239, Training Accuracy= 0.09375
Iter 4480, Minibatch Loss= 2.330398, Training Accuracy= 0.08594
Iter 5120, Minibatch Loss= 2.316712, Training Accuracy= 0.13281
Iter 5760, Minibatch Loss= 2.343754, Training Accuracy= 0.10156
Iter 6400, Minibatch Loss= 2.318546, Training Accuracy= 0.16406
Iter 7040, Minibatch Loss= 2.305453, Training Accuracy= 0.08594
Iter 7680, Minibatch Loss= 2.291348, Training Accuracy= 0.13281
Iter 8320, Minibatch Loss= 2.251736, Training Accuracy= 0.21094
Iter 8960, Minibatch Loss= 2.216596, Training Accuracy= 0.26562
Iter 9600, Minibatch Loss= 2.132652, Training Accuracy= 0.24219
Iter 10240, Minibatch Loss= 1.907876, Tra